# Creating spreadsheet from B3 to income tax

# Criando uma planilha com dados retirados da B3 para declarar no imposto do renda

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

Is necessary to put the data in raw_data directory because the informations are personas

Necessário colocar os dados em raw_data por se tratar de informações pessoais

In [2]:
path = '../raw_data'
files = [f for f in listdir(path) if isfile(join(path, f))] 

### Feature engineering

### Organizando as colunas 

In [3]:
list_files = []
for file in range(0, len(files)):
    list_file = pd.read_excel(f'{path}/{files[file]}', parse_dates=['Data'])
    list_files.append(list_file)
list_all_files = pd.concat(list_files)
list_files_s = list_all_files.sort_values(by=['Produto', 'Data'])
df_files = pd.DataFrame(list_files_s)
df_files

,Entrada/Saída,Data,Movimentação,Produto,Instituição,Quantidade,Preço unitário,Valor da Operação
63,Credito,2020-04-17,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,10,24,240
62,Credito,2020-04-20,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,19,24.5,465.5
47,Debito,2020-05-19,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,34,20.82,707.88
85,Credito,2020-06-01,Transferência - Liquidação,ALSO3 - ALIANSCE SONAE SHOPPING CENTERS S.A. ...,CLEAR CORRETORA - GRUPO XP,5,50.5,252.5
38,Debito,2020-06-25,Transferência - Liquidação,ALUP11 - ALUPAR INVESTIMENTO S/A ...,CLEAR CORRETORA - GRUPO XP,35,24.45,855.75
...,...,...,...,...,...,...,...,...
109,Credito,2020-10-12,Dividendo,YDUQ3 - YDUQS PARTICIPACOES S.A,CLEAR CORRETORA - GRUPO XP,30,0.51,15.29
38,Credito,2021-07-12,Dividendo,YDUQ3 - YDUQS PARTICIPACOES S.A,CLEAR CORRETORA - GRUPO XP,30,0.47,14.09
16,Debito,2021-09-22,Transferência - Liquidação,YDUQ3 - YDUQS PARTICIPACOES S.A,CLEAR CORRETORA - GRUPO XP,30,23.64,709.2
66,Credito,2020-02-04,Transferência - Liquidação,YDUQ3 - ESTACIO PARTICIPACOES S.A. ...,CLEAR CORRETORA - GRUPO XP,30,23,690


In [4]:
df_files['Data da compra'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Credito')
df_files['Data da venda'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Debito')
df_files = df_files[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Movimentação', 'Instituição']]
df_files['Quantidade'] = df_files['Quantidade'].apply(lambda x: x.replace(',','.'))
df_files['Quantidade'] = df_files['Quantidade'].astype(float)
df_files.tail(15)

/tmp/ipykernel_7973/4220225477.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_files['Quantidade'] = df_files['Quantidade'].apply(lambda x: x.replace(',','.'))
/tmp/ipykernel_7973/4220225477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_files['Quantidade'] = df_files['Quantidade'].astype(float)


,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Movimentação,Instituição
92,VISC11 - VINCI SHOPPING CENTERS FDO INVEST IMO...,2021-12-02,NaT,27.0,0.6,16.2,Rendimento,CLEAR CORRETORA - GRUPO XP
84,VISC11 - VINCI SHOPPING CENTERS FDO INVEST IMO...,2021-12-03,NaT,27.0,0.55,14.85,Rendimento,CLEAR CORRETORA - GRUPO XP
22,VISC12 - VINCI SHOPPING CENTERS FI IMOBI...,NaT,2021-08-24,8.0,-,-,Direitos de Subscrição - Não Exercido,CLEAR CORRETORA - GRUPO XP
26,VISC12 - VINCI SHOPPING CENTERS FDO INVEST IMO...,2021-10-08,NaT,8.0,-,-,Direito de Subscrição,CLEAR CORRETORA - GRUPO XP
98,VULC3 - VULCABRAS/AZALEIA S.A. ...,NaT,2019-05-11,161.0,8.03,1292.83,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP
75,VVAR3 - VIA VAREJO S.A. ...,NaT,2020-03-17,190.0,8.8,1672,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP
73,VVAR3 - VIA VAREJO S.A. ...,2020-03-18,NaT,100.0,7.56,756,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP
54,VVAR3 - VIA VAREJO S.A. ...,NaT,2020-05-13,100.0,8.45,845,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP
78,VVAR3 - VIA VAREJO S.A. ...,2020-12-03,NaT,100.0,10,1000,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP
19,YDUQ3 - YDUQS PARTICIPACOES S.A,2020-08-31,NaT,25.0,27.85,696.25,Transferência - Liquidação,CLEAR CORRETORA - GRUPO XP


## Fixed income

In [5]:
df_fixedi_c = df_files[df_files['Movimentação'] == 'Compra']
df_fixedi_v = df_files[df_files['Movimentação'] == 'Venda']
df_fixedi = pd.concat([df_fixedi_c, df_fixedi_v])
df_fixedi = df_fixedi[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Instituição']]
df_fixedi = df_fixedi.sort_values(by=['Produto']).reset_index()
df_fixedi

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição
0,92,Tesouro IPCA+ 2035,NaT,2019-11-27,1.57,1928.14,3027.18,XP INVESTIMENTOS CCTVM S/A
1,11,Tesouro IPCA+ com Juros Semestrais 2055,2020-09-17,NaT,0.44,4527.61,1992.15,CLEAR CORRETORA - GRUPO XP
2,7,Tesouro IPCA+ com Juros Semestrais 2055,NaT,2021-10-18,0.44,4181.39,1839.81,CLEAR CORRETORA - GRUPO XP
3,93,Tesouro Prefixado 2025,NaT,2019-11-27,3.71,718.32,2664.97,XP INVESTIMENTOS CCTVM S/A
4,94,Tesouro Selic 2023,NaT,2019-11-27,0.18,10420.13,1875.62,XP INVESTIMENTOS CCTVM S/A


In [6]:
df_fixedi_buy = df_fixedi[df_fixedi['Data da compra'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).reset_index()
df_fixedi_sel = df_fixedi[df_fixedi['Data da venda'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).transform(lambda x: x *-1).reset_index()

quantidades = pd.concat([df_fixedi_sel, df_fixedi_buy]).reset_index()
quantidades_prod = quantidades.groupby(['Produto']).sum().reset_index()
quantidades_prod.drop(columns='index', inplace=True)
quantidades_prod

,Produto,Quantidade
0,Tesouro IPCA+ 2035,-1.57
1,Tesouro IPCA+ com Juros Semestrais 2055,0.00
2,Tesouro Prefixado 2025,-3.71
3,Tesouro Selic 2023,-0.18


In [7]:
for line in range(0, len(quantidades_prod)):
    if quantidades_prod.Quantidade[line] < 0:
        quantidades_prod.Quantidade[line] = quantidades_prod.Quantidade[line]*-1
    else:
        quantidades_prod.Quantidade[line]
        
quantidades_prod = quantidades_prod.rename(columns={'Quantidade': 'Saldo no Tesouro'})
quantidades_prod = pd.DataFrame(quantidades_prod)
quantidades_prod

/tmp/ipykernel_7973/1494921743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quantidades_prod.Quantidade[line] = quantidades_prod.Quantidade[line]*-1


,Produto,Saldo no Tesouro
0,Tesouro IPCA+ 2035,1.57
1,Tesouro IPCA+ com Juros Semestrais 2055,0.00
2,Tesouro Prefixado 2025,3.71
3,Tesouro Selic 2023,0.18


In [8]:
df_fixedi_opB = df_fixedi[df_fixedi['Data da compra'] > '20000101'].reset_index()
df_fixedi_opS = df_fixedi[df_fixedi['Data da venda'] > '20000101'].reset_index()

df_fixedi_opB['Lucro/Prejuízo'] = 0

for lineS in range(0, len(df_fixedi_opS)):
    for lineB in range(0, len(df_fixedi_opB)):
        if df_fixedi_opS['Quantidade'][lineS] == df_fixedi_opB['Quantidade'][lineB]:
            df_fixedi_opB['Lucro/Prejuízo'][lineB] = df_fixedi_opS['Valor da Operação'][lineS] - df_fixedi_opB['Valor da Operação'][lineB]
df_fixedi_opB = df_fixedi_opB[['Produto', 'Lucro/Prejuízo']]

df_fixedi_opB

/tmp/ipykernel_7973/2543932147.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixedi_opB['Lucro/Prejuízo'][lineB] = df_fixedi_opS['Valor da Operação'][lineS] - df_fixedi_opB['Valor da Operação'][lineB]


,Produto,Lucro/Prejuízo
0,Tesouro IPCA+ com Juros Semestrais 2055,-152


In [9]:
sald_lu = pd.concat([quantidades_prod, df_fixedi_opB]).reset_index()
sald_lu

,index,Produto,Saldo no Tesouro,Lucro/Prejuízo
0,0,Tesouro IPCA+ 2035,1.57,NaN
1,1,Tesouro IPCA+ com Juros Semestrais 2055,0.00,NaN
2,2,Tesouro Prefixado 2025,3.71,NaN
3,3,Tesouro Selic 2023,0.18,NaN
4,0,Tesouro IPCA+ com Juros Semestrais 2055,NaN,-152.0


In [10]:
lu_prej = sald_lu.groupby(['Produto']).sum().reset_index()
lu_prej = lu_prej.drop(columns='index')

lu_prej

,Produto,Saldo no Tesouro,Lucro/Prejuízo
0,Tesouro IPCA+ 2035,1.57,0.0
1,Tesouro IPCA+ com Juros Semestrais 2055,0.00,-152.0
2,Tesouro Prefixado 2025,3.71,0.0
3,Tesouro Selic 2023,0.18,0.0


In [11]:
merge_fixo = pd.merge(df_fixedi, lu_prej, how='left', on='Produto')
merge_fixo

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição,Saldo no Tesouro,Lucro/Prejuízo
0,92,Tesouro IPCA+ 2035,NaT,2019-11-27,1.57,1928.14,3027.18,XP INVESTIMENTOS CCTVM S/A,1.57,0.0
1,11,Tesouro IPCA+ com Juros Semestrais 2055,2020-09-17,NaT,0.44,4527.61,1992.15,CLEAR CORRETORA - GRUPO XP,0.00,-152.0
2,7,Tesouro IPCA+ com Juros Semestrais 2055,NaT,2021-10-18,0.44,4181.39,1839.81,CLEAR CORRETORA - GRUPO XP,0.00,-152.0
3,93,Tesouro Prefixado 2025,NaT,2019-11-27,3.71,718.32,2664.97,XP INVESTIMENTOS CCTVM S/A,3.71,0.0
4,94,Tesouro Selic 2023,NaT,2019-11-27,0.18,10420.13,1875.62,XP INVESTIMENTOS CCTVM S/A,0.18,0.0


In [12]:
merge_fixo.drop(columns=['index', 'Instituição'], inplace = True)
merge_fixo

,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Saldo no Tesouro,Lucro/Prejuízo
0,Tesouro IPCA+ 2035,NaT,2019-11-27,1.57,1928.14,3027.18,1.57,0.0
1,Tesouro IPCA+ com Juros Semestrais 2055,2020-09-17,NaT,0.44,4527.61,1992.15,0.00,-152.0
2,Tesouro IPCA+ com Juros Semestrais 2055,NaT,2021-10-18,0.44,4181.39,1839.81,0.00,-152.0
3,Tesouro Prefixado 2025,NaT,2019-11-27,3.71,718.32,2664.97,3.71,0.0
4,Tesouro Selic 2023,NaT,2019-11-27,0.18,10420.13,1875.62,0.18,0.0


In [13]:
merge_fixo = merge_fixo.sort_values(['Produto', 'Data da compra', 'Data da venda'])
merge_fixo

,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Saldo no Tesouro,Lucro/Prejuízo
0,Tesouro IPCA+ 2035,NaT,2019-11-27,1.57,1928.14,3027.18,1.57,0.0
1,Tesouro IPCA+ com Juros Semestrais 2055,2020-09-17,NaT,0.44,4527.61,1992.15,0.00,-152.0
2,Tesouro IPCA+ com Juros Semestrais 2055,NaT,2021-10-18,0.44,4181.39,1839.81,0.00,-152.0
3,Tesouro Prefixado 2025,NaT,2019-11-27,3.71,718.32,2664.97,3.71,0.0
4,Tesouro Selic 2023,NaT,2019-11-27,0.18,10420.13,1875.62,0.18,0.0


In [14]:
merge_fixo['Lucro/Prejuizo'] = ''
merge_fixo['Saldo Tesouro'] = ''

for line in range(0, len(merge_fixo)-1):
    if merge_fixo['Produto'].iloc[line] != merge_fixo['Produto'].iloc[line+1]:
        merge_fixo['Lucro/Prejuizo'].iloc[line] = merge_fixo['Lucro/Prejuízo'].iloc[line]
        merge_fixo['Saldo Tesouro'].iloc[line] = merge_fixo['Saldo no Tesouro'].iloc[line]
    else:
        merge_fixo['Lucro/Prejuizo'].iloc[line] = 'nan'
        merge_fixo['Saldo Tesouro'].iloc[line] = 'nan'
merge_fixo['Lucro/Prejuizo'].iloc[len(merge_fixo)-1] = merge_fixo['Lucro/Prejuízo'].iloc[len(merge_fixo)-1]
merge_fixo['Saldo Tesouro'].iloc[len(merge_fixo)-1] = merge_fixo['Saldo no Tesouro'].iloc[len(merge_fixo)-1]
merge_fixo.drop(columns=['Saldo no Tesouro', 'Lucro/Prejuízo'], inplace=True)
merge_fixo

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Lucro/Prejuizo,Saldo Tesouro
0,Tesouro IPCA+ 2035,NaT,2019-11-27,1.57,1928.14,3027.18,0.0,1.57
1,Tesouro IPCA+ com Juros Semestrais 2055,2020-09-17,NaT,0.44,4527.61,1992.15,nan,nan
2,Tesouro IPCA+ com Juros Semestrais 2055,NaT,2021-10-18,0.44,4181.39,1839.81,-152.0,0.0
3,Tesouro Prefixado 2025,NaT,2019-11-27,3.71,718.32,2664.97,0.0,3.71
4,Tesouro Selic 2023,NaT,2019-11-27,0.18,10420.13,1875.62,0.0,0.18


## Variable Income

In [15]:
df_stock = df_files[df_files['Movimentação'] == 'Transferência - Liquidação']
df_stock = df_stock[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação']]
df_stock = df_stock.sort_values(by=['Produto', 'Data da compra', 'Data da venda']).reset_index()
df_stock['Produto'] = df_stock['Produto'].apply(lambda x: x.split('-')[0].strip())
df_stock.head(10)

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação
0,63,ALSO3,2020-04-17,NaT,10.0,24,240
1,62,ALSO3,2020-04-20,NaT,19.0,24.5,465.5
2,85,ALSO3,2020-06-01,NaT,5.0,50.5,252.5
3,47,ALSO3,NaT,2020-05-19,34.0,20.82,707.88
4,55,ALUP11,2020-11-05,NaT,75.0,21.95,1646.25
5,38,ALUP11,NaT,2020-06-25,35.0,24.45,855.75
6,27,ALUP11,NaT,2020-07-30,40.0,24.24,969.6
7,27,AMAR3,2021-07-27,NaT,100.0,8.18,818
8,71,AZUL4,2020-03-18,NaT,100.0,16,1600
9,2,B3SA3,2020-10-14,NaT,4.0,54.5,218


In [16]:
df_stock_buy = df_stock[df_stock['Data da compra'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).reset_index()
df_stock_buy

,Produto,Quantidade
0,ALSO3,34.0
1,ALUP11,75.0
2,AMAR3,100.0
3,AZUL4,100.0
4,B3SA3,19.0
5,BPAC11,51.0
6,BPAN4,400.0
7,BRBI11,22.0
8,BRFS3,30.0
9,BRKM5,55.0


In [17]:
df_stock_sel = df_stock[df_stock['Data da venda'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).transform(lambda x: x *-1).reset_index()
df_stock_sel

,Produto,Quantidade
0,ALSO3,-34.0
1,ALUP11,-75.0
2,BPAN4,-40.0
3,BRFS3,-30.0
4,BRKM5,-55.0
5,BRSR6,-70.0
6,CSNA3,-48.0
7,CVCB3,-100.0
8,ELET3,-25.0
9,ENEV3,-100.0


In [18]:
quantidades_stock = pd.concat([df_stock_sel, df_stock_buy]).reset_index()
quantidades_prod_stock = quantidades_stock.groupby(['Produto']).sum().reset_index()
quantidades_prod_stock

,Produto,index,Quantidade
0,ALSO3,0,0.0
1,ALUP11,2,0.0
2,AMAR3,2,100.0
3,AZUL4,3,100.0
4,B3SA3,4,19.0
5,BPAC11,5,51.0
6,BPAN4,8,360.0
7,BRBI11,7,22.0
8,BRFS3,11,0.0
9,BRKM5,13,0.0


In [19]:
# for line in range(0, len(quantidades_prod_stock)):
#     if quantidades_prod_stock.Quantidade[line] < 0:
#         quantidades_prod_stock.Quantidade[line] = quantidades_prod_stock.Quantidade[line]*-1
#     else:
#         quantidades_prod_stock.Quantidade[line]
# quantidades_prod_stock

In [20]:
quantidades_prod_stock = pd.DataFrame(quantidades_prod_stock)
quantidades_prod_stock = quantidades_prod_stock.drop(columns='index')
quantidades_prod_stock = quantidades_prod_stock.rename(columns={'Quantidade': 'Saldo de ações'})
quantidades_prod_stock

,Produto,Saldo de ações
0,ALSO3,0.0
1,ALUP11,0.0
2,AMAR3,100.0
3,AZUL4,100.0
4,B3SA3,19.0
5,BPAC11,51.0
6,BPAN4,360.0
7,BRBI11,22.0
8,BRFS3,0.0
9,BRKM5,0.0


In [21]:
df_stock_opB = df_stock[df_stock['Data da compra'] > '20000101'].sort_values(['Produto', 'Data da compra']).reset_index()
df_stock_opB.head()

,level_0,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação
0,0,63,ALSO3,2020-04-17,NaT,10.0,24,240
1,1,62,ALSO3,2020-04-20,NaT,19.0,24.5,465.5
2,2,85,ALSO3,2020-06-01,NaT,5.0,50.5,252.5
3,4,55,ALUP11,2020-11-05,NaT,75.0,21.95,1646.25
4,7,27,AMAR3,2021-07-27,NaT,100.0,8.18,818


In [22]:
df_stock_opS = df_stock[df_stock['Data da venda'] > '20000101'].sort_values(['Produto', 'Data da venda']).reset_index()
df_stock_opS.head()

,level_0,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação
0,3,47,ALSO3,NaT,2020-05-19,34.0,20.82,707.88
1,5,38,ALUP11,NaT,2020-06-25,35.0,24.45,855.75
2,6,27,ALUP11,NaT,2020-07-30,40.0,24.24,969.6
3,14,12,BPAN4,NaT,2021-09-22,40.0,15.81,632.4
4,19,17,BRFS3,NaT,2020-08-31,30.0,19.9,597


In [23]:
data_buy = []
data_sell = []
for lineB in range(0, len(df_stock_opB)):
    for lineS in range(0, len(df_stock_opS)):
        if df_stock_opS.Produto[lineS] == df_stock_opB.Produto[lineB] and df_stock_opS['Data da venda'][lineS] >= df_stock_opB['Data da compra'][lineB]:
            buy = df_stock_opB['Valor da Operação'][lineB]
            sell = df_stock_opS['Valor da Operação'][lineS]
            prod = df_stock_opB['Produto'][lineB]
            datb = df_stock_opB['Data da compra'][lineB]
            dats = df_stock_opS['Data da venda'][lineS]
            data_buy.append([datb, prod, buy])
            data_sell.append([dats, prod, sell])
        if df_stock_opS.Produto[lineS] == df_stock_opB.Produto[lineB] and df_stock_opS['Data da venda'][lineS] <= df_stock_opB['Data da compra'][lineB]:
            sell2 = df_stock_opS['Valor da Operação'][lineS]
            prod2 = df_stock_opB['Produto'][lineB]
            dats2 = df_stock_opS['Data da venda'][lineS]
            data_sell.append([dats2, prod2, sell2])
buy_stock = pd.DataFrame(data_buy, columns=['Data da compra', 'Produto', 'Compra'])           
sell_stock = pd.DataFrame(data_sell, columns=['Data da venda', 'Produto', 'Venda']) 
sell_stock = sell_stock.drop_duplicates()
print(sell_stock.shape)
print(buy_stock.shape)

(23, 3)
(26, 3)


In [24]:
df_stock_buy = buy_stock.groupby('Produto').sum().reset_index()
df_stock_buy

,Produto,Compra
0,ALSO3,705.50
1,BPAN4,2591.00
2,BRFS3,681.00
3,BRKM5,1133.00
4,BRSR6,1013.60
5,CVCB3,950.00
6,ELET3,600.00
7,ENEV3,738.75
8,HYPE3,810.00
9,ITUB4,2300.00


In [25]:
sell_stock = sell_stock.drop(columns=['Data da venda'])
sell_stock

,Produto,Venda
0,ALSO3,707.88
3,ALUP11,855.75
4,ALUP11,969.60
5,BPAN4,632.40
8,BRFS3,597.00
9,BRKM5,1518.00
10,BRSR6,963.90
11,CSNA3,341.76
14,CVCB3,1090.00
15,ELET3,550.00


In [26]:
lucpre_stock = pd.concat([sell_stock, df_stock_buy]).reset_index()
lucpre_stock

,index,Produto,Venda,Compra
0,0,ALSO3,707.88,NaN
1,3,ALUP11,855.75,NaN
2,4,ALUP11,969.60,NaN
3,5,BPAN4,632.40,NaN
4,8,BRFS3,597.00,NaN
5,9,BRKM5,1518.00,NaN
6,10,BRSR6,963.90,NaN
7,11,CSNA3,341.76,NaN
8,14,CVCB3,1090.00,NaN
9,15,ELET3,550.00,NaN


In [27]:
# ir_stock = lucpre_stock.groupby(['Produto']).sum().reset_index()
# ir_stock
df_stock_opB = df_stock_opB[['Produto', 'level_0', 'index', 'Quantidade', 'Preço unitário', 'Valor da Operação']]
df_stock_opB

,Produto,level_0,index,Quantidade,Preço unitário,Valor da Operação
0,ALSO3,0,63,10.0,24,240
1,ALSO3,1,62,19.0,24.5,465.5
2,ALSO3,2,85,5.0,50.5,252.5
3,ALUP11,4,55,75.0,21.95,1646.25
4,AMAR3,7,27,100.0,8.18,818
...,...,...,...,...,...,...
76,VISC11,97,42,27.0,110.2,2975.4
77,VVAR3,99,73,100.0,7.56,756
78,VVAR3,100,78,100.0,10,1000
79,YDUQ3,105,66,30.0,23,690


In [28]:
df_stock_opS = df_stock_opS[['Produto', 'level_0', 'index', 'Quantidade', 'Preço unitário', 'Valor da Operação']]
df_stock_opS

,Produto,level_0,index,Quantidade,Preço unitário,Valor da Operação
0,ALSO3,3,47,34.0,20.82,707.88
1,ALUP11,5,38,35.0,24.45,855.75
2,ALUP11,6,27,40.0,24.24,969.6
3,BPAN4,14,12,40.0,15.81,632.4
4,BRFS3,19,17,30.0,19.9,597
5,BRKM5,22,43,55.0,27.6,1518
6,BRSR6,24,89,70.0,13.77,963.9
7,CSNA3,32,59,48.0,7.12,341.76
8,CVCB3,34,52,100.0,10.9,1090
9,ELET3,37,49,25.0,22,550


In [29]:
df_stock_opB = df_stock_opB.groupby('Produto').sum().reset_index()
df_stock_opB

,Produto,level_0,index,Quantidade
0,ALSO3,3,210,34.0
1,ALUP11,4,55,75.0
2,AMAR3,7,27,100.0
3,AZUL4,8,71,100.0
4,B3SA3,19,34,19.0
5,BPAC11,36,38,51.0
6,BPAN4,48,232,400.0
7,BRBI11,18,34,22.0
8,BRFS3,20,48,30.0
9,BRKM5,21,61,55.0


In [32]:
df_stock_opS = df_stock_opS.groupby('Produto').sum().reset_index()
df_stock_opS

,Produto,level_0,index,Quantidade
0,ALSO3,3,47,34.0
1,ALUP11,11,65,75.0
2,BPAN4,14,12,40.0
3,BRFS3,19,17,30.0
4,BRKM5,22,43,55.0
5,BRSR6,24,89,70.0
6,CSNA3,32,59,48.0
7,CVCB3,34,52,100.0
8,ELET3,37,49,25.0
9,ENEV3,40,74,100.0


In [31]:
calcu = []
for lineB in range(0, len(df_stock_opB)):
    for lineS in range(0, len(df_stock_opS)):
        if df_stock_opS.Produto[lineS] == df_stock_opB.Produto[lineB]:
            prod = df_stock_opB.Produto[lineB]
            cal = df_stock_opS['Valor da Operação'][lineS]-((df_stock_opB['Valor da Operação'][lineB]/df_stock_opB.Quantidade[lineB])*df_stock_opS.Quantidade[lineS])
            calcu.append([prod, cal])
                
calcula = pd.DataFrame(calcu, columns=['Produto', 'Lucro/Prejuízo']) 
calcula

KeyError: 'Valor da Operação'

In [ ]:
stock_i = pd.merge(quantidades_prod_stock, calcula, how='left', on='Produto')
stock_i

In [ ]:
merge_stock = pd.merge(df_stock, stock_i, how='left', on='Produto')
merge_stock.drop(columns='index', inplace = True)
merge_stock = merge_stock.sort_values(['Produto', 'Data da compra', 'Data da venda'])
merge_stock

In [ ]:
merge_stock['Lucro/Prejuizo'] = ''
merge_stock['Saldo das ações'] = ''

for line in range(0, len(merge_stock)-1):
    if merge_stock['Produto'].iloc[line] != merge_stock['Produto'].iloc[line+1]:
        merge_stock['Lucro/Prejuizo'].iloc[line] = merge_stock['Lucro/Prejuízo'].iloc[line]
        merge_stock['Saldo das ações'].iloc[line] = merge_stock['Saldo de ações'].iloc[line]
    else:
        merge_stock['Lucro/Prejuizo'].iloc[line] = 'nan'
        merge_stock['Saldo das ações'].iloc[line] = 'nan'
merge_stock['Lucro/Prejuizo'].iloc[len(merge_stock)-1] = merge_stock['Lucro/Prejuízo'].iloc[len(merge_stock)-1]
merge_stock['Saldo das ações'].iloc[len(merge_stock)-1] = merge_stock['Saldo de ações'].iloc[len(merge_stock)-1]
merge_stock.drop(columns=['Saldo de ações', 'Lucro/Prejuízo'], inplace=True)
merge_stock.tail(15)

### Se produto for comprado depois de vender

In [ ]:
for line in range(0, len(merge_stock)-1):
    if merge_stock.Produto[line] == merge_stock.Produto[line+1] and merge_stock['Data da venda'][line] <= merge_stock['Data da compra'][line+1]:
        print(merge_stock.Produto[line])